In [1]:
import os
import numpy as np
import math

from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image

import torch.nn as nn
import torch.nn.functional as F
import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

torch.manual_seed(10)

In [2]:
# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(28), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=4096,
    shuffle=True,
)

In [3]:
class Generator(nn.Module): 
    def __init__(self):
        super().__init__()
        
        channels, img_size = 1, 28
        latent_dim = 100
        self.image_shape = (channels, img_size, img_size)
        
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        
        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),   # * means unpacking (list, tuple)
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, np.prod(self.image_shape)),   # 784
            nn.Tanh()
        )
    
    def forward(self, x):
        img = self.model(x)
        img = img.view(img.size(0), *self.image_shape)
        return img
    
    

In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

In [5]:
cuda = True if torch.cuda.is_available() else False
n_epochs = 200
lr = 0.0002
b1 = 0.5
b2 = 0.999


G = Generator()
D = Discriminator()

loss_function = nn.BCELoss()


if cuda:
    G.cuda()
    D.cuda()
    device = torch.device('cuda')
    
optim_G = torch.optim.Adam(G.parameters(), lr=lr, betas=(b1, b2))
optim_D = torch.optim.Adam(D.parameters(), lr=lr, betas=(b1, b2))


In [ ]:
for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # Adversarial ground truths
        valid = torch.ones(imgs.size(0),1, requires_grad=False, device=device)
        fake = torch.zeros(imgs.size(0),1, requires_grad=False, device=device)
        
        real_imgs = imgs.type(torch.cuda.FloatTensor)
        
        # Train Generator
        optim_G.zero_grad()
        z = torch.randn([imgs.shape[0], 100], device=device)   # 100: latent_dim
        gen_imgs = G(z)
        g_loss = loss_function(D(gen_imgs), valid)   # Discriminator를 진짜라고 속이는 방향으로 학습
        
        g_loss.backward()
        optim_G.step()
        
        # Train Discriminator
        optim_D.zero_grad()
        
        real_loss = loss_function(D(real_imgs), valid)
        fake_loss = loss_function(D(gen_imgs.detach()), fake)
        
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optim_D.step()
        
        
    print(
        "[Epoch %d/%d] [D loss: %f] [G loss: %f]"
        % (epoch, n_epochs, d_loss.item(), g_loss.item())
    )

    
    if (epoch+1) % 10 == 0:
        save_image(gen_imgs.data[:25], f"../../data/gan/{epoch+1}.png", nrow=5, normalize=True)

[Epoch 0/200] [D loss: 0.423126] [G loss: 0.577006]
[Epoch 1/200] [D loss: 0.503467] [G loss: 0.608587]
[Epoch 2/200] [D loss: 0.460986] [G loss: 0.753251]
[Epoch 3/200] [D loss: 0.478365] [G loss: 0.756156]
[Epoch 4/200] [D loss: 0.553855] [G loss: 0.609741]
[Epoch 5/200] [D loss: 0.361735] [G loss: 0.949578]
[Epoch 6/200] [D loss: 0.491487] [G loss: 0.758932]
[Epoch 7/200] [D loss: 0.561717] [G loss: 0.657450]
[Epoch 8/200] [D loss: 0.691802] [G loss: 0.628700]
[Epoch 9/200] [D loss: 0.613466] [G loss: 0.643305]
[Epoch 10/200] [D loss: 0.656488] [G loss: 0.556321]
[Epoch 11/200] [D loss: 0.712181] [G loss: 0.696349]
[Epoch 12/200] [D loss: 0.626507] [G loss: 0.641300]
[Epoch 13/200] [D loss: 0.619393] [G loss: 0.657371]
[Epoch 14/200] [D loss: 0.566474] [G loss: 1.274426]
[Epoch 15/200] [D loss: 0.443301] [G loss: 0.940071]
[Epoch 16/200] [D loss: 0.457194] [G loss: 0.918292]
[Epoch 17/200] [D loss: 0.467228] [G loss: 1.117216]
[Epoch 18/200] [D loss: 0.506735] [G loss: 0.880343]
[Ep